In [1]:
!pip install git+https://github.com/JohanLokna/DeNovoProteinsPairsGNN.git --upgrade

  Cloning https://github.com/JohanLokna/DeNovoProteinsPairsGNN.git to /tmp/pip-req-build-xzhg7o3y
  Running command git clone -q https://github.com/JohanLokna/DeNovoProteinsPairsGNN.git /tmp/pip-req-build-xzhg7o3y
  Created wheel for DeNovoProteinsPairsGNN: filename=DeNovoProteinsPairsGNN-0.0.1-cp37-none-any.whl size=20644 sha256=066d9673bd58938bbb4993dc41253c61ab8fda71e2861b19d94eb462d247cf5c
  Stored in directory: /tmp/pip-ephem-wheel-cache-79zw92p8/wheels/53/f0/7f/a3bba6c99f3e23a271efe6d5b4ec0bfa667ba78e1f4c3e2e1e
Successfully built DeNovoProteinsPairsGNN
  Found existing installation: DeNovoProteinsPairsGNN 0.0.1
    Uninstalling DeNovoProteinsPairsGNN-0.0.1:
      Successfully uninstalled DeNovoProteinsPairsGNN-0.0.1


In [2]:
from google.colab import drive

drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
from ProteinPairsGenerator.models import Net
import torch

batch_size = 1
num_features = 20
adj_input_size = 2
hidden_size = 128
frac_present = 0.5
frac_present_valid = frac_present
info_size= 1024

device = "cuda" if torch.cuda.is_available() else "cpu"

state_file = "/content/gdrive/My Drive/e53-s1952148-d93703104.state"

net = Net(
    x_input_size=num_features + 1, adj_input_size=adj_input_size, hidden_size=hidden_size, output_size=num_features
)

net.load_state_dict(torch.load(state_file, map_location=device))
net.eval()
net = net.to(device)

In [4]:
import tempfile
from pathlib import Path 

from ProteinPairsGenerator.datasets import ProteinInMemoryDataset

datasets = {}

DATA_ROOT = Path(tempfile.gettempdir())
DATA_ROOT = Path(tempfile.gettempdir()).joinpath("ml_data")
DATA_ROOT.mkdir(exist_ok=True)

data_file = '/content/gdrive/My Drive/part-00000-4f535e50-cdf4-4275-b6b3-a3038f24a1a9-c000.snappy.parquet'

datasets["protein_test"] = ProteinInMemoryDataset(
    root=DATA_ROOT / "protein_test",
    subset="test",
    data_url=data_file,
)

Processing...


/usr/local/lib/python3.7/dist-packages/ProteinPairsGenerator/datasets/in_memory.py:134: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(data_array)


Done!


In [ ]:
from ProteinPairsGenerator.utils import design_protein
from torch_geometric.data import DataLoader

for frac_present in [0.0, 0.5, 0.8]:
    results = []
    for i, data in enumerate(DataLoader(datasets["protein_test"], shuffle=False, num_workers=1, batch_size=1, drop_last=False)):
        data.to(device)
        if frac_present == 0:
            is_present = torch.zeros(data.x.size(0), dtype=torch.bool, device=data.x.device)
            x_in = torch.ones_like(data.x) * 20
        else:
            is_present = torch.rand(data.x.size(0), device=data.x.device) < frac_present
            x_in = torch.where(is_present, data.x, torch.tensor(20, device=data.x.device))
        x_pred = design_protein(net, x_in, data.edge_index, data.edge_attr)
        identity_all = float((x_pred == data.x).sum()) / data.x.size(0)
        identity_missing = float((x_pred[~is_present] == data.x[~is_present]).sum()) / (~is_present).sum().item()
        result = {
            "identity_all": identity_all,
            "identity_missing": identity_missing,
            "num_correct": (x_pred[~is_present] == data.x[~is_present]).sum().item(),
            "num_total": (~is_present).sum().item(),
        }
        results.append(result)
        print(result, '\n\n')

{'identity_all': 0.31390134529147984, 'identity_missing': 0.31390134529147984, 'num_correct': 70, 'num_total': 223} 


{'identity_all': 0.46017699115044247, 'identity_missing': 0.46017699115044247, 'num_correct': 52, 'num_total': 113} 


{'identity_all': 0.38823529411764707, 'identity_missing': 0.38823529411764707, 'num_correct': 33, 'num_total': 85} 


{'identity_all': 0.27848101265822783, 'identity_missing': 0.27848101265822783, 'num_correct': 22, 'num_total': 79} 


{'identity_all': 0.1076923076923077, 'identity_missing': 0.1076923076923077, 'num_correct': 7, 'num_total': 65} 


{'identity_all': 0.28169014084507044, 'identity_missing': 0.28169014084507044, 'num_correct': 20, 'num_total': 71} 


{'identity_all': 0.43670886075949367, 'identity_missing': 0.43670886075949367, 'num_correct': 69, 'num_total': 158} 


{'identity_all': 0.35789473684210527, 'identity_missing': 0.35789473684210527, 'num_correct': 34, 'num_total': 95} 


{'identity_all': 0.0576271186440678, 'identity_missing':